In [66]:
import pandas as pd

In [67]:
# Pulled dataset from HF, dropped unneeded index column
df = pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv").drop(
    "Unnamed: 0", axis=1
)
df["duration_s"] = df["duration_ms"] / 1000
df_nodupe = df.drop_duplicates(subset=["track_id"])

In [68]:
df.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,duration_s
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,...,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,230.666
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,...,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,149.610
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,...,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,210.826
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,...,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,201.933
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,...,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,198.853


In [69]:
g_dummy = pd.get_dummies(df['track_genre']).groupby(df['track_id']).sum().astype(int).reset_index()

In [70]:
dummy_val = g_dummy.copy()
dummy_val["total"] = dummy_val.sum(axis=1, numeric_only=True)
dummy_val = dummy_val[["track_id", "total"]].sort_values("track_id", ascending=True)

process_check = (
    df.groupby("track_id").size().to_frame("total").reset_index().sort_values("track_id", ascending=True)
)

for df1, df2 in zip(process_check.iterrows(), dummy_val.iterrows(), strict=True):
    assert (df1[1]["total"] == df2[1]["total"]) and (df1[1]["track_id"] == df2[1]["track_id"])

dummy_val

,track_id,total
0,0000vdREvCVMxbQTkS888c,1
1,000CC8EParg64OmTxVnZ0p,1
2,000Iz0K615UepwSJ5z2RE5,1
3,000RDCYioLteXcutOjeweY,1
4,000qpdoc97IMTBvF8gwcpy,1
...,...,...
89736,7zxHiMmVLt4LGWpOMqOpUh,1
89737,7zxpdh3EqMq2JCkOI0EqcG,1
89738,7zyYmIdjqqiX6kLryb7QBx,1
89739,7zybSU9tFO9HNlwmGF7stc,1


In [71]:
df = df_nodupe.merge(g_dummy, on="track_id").drop(
    ["track_id", "artists", "album_name", "track_name", "track_genre"], axis=1
)
df["explicit"] = df["explicit"].astype(int)
df

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,...,spanish,study,swedish,synth-pop,tango,techno,trance,trip-hop,turkish,world-music
0,73,230666,0,0.676,0.4610,1,-6.746,0,0.1430,0.0322,...,0,0,0,0,0,0,0,0,0,0
1,55,149610,0,0.420,0.1660,1,-17.235,1,0.0763,0.9240,...,0,0,0,0,0,0,0,0,0,0
2,57,210826,0,0.438,0.3590,0,-9.734,1,0.0557,0.2100,...,0,0,0,0,0,0,0,0,0,0
3,71,201933,0,0.266,0.0596,0,-18.515,1,0.0363,0.9050,...,0,0,0,0,0,0,0,0,0,0
4,82,198853,0,0.618,0.4430,2,-9.681,1,0.0526,0.4690,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89736,21,384999,0,0.172,0.2350,5,-16.393,1,0.0422,0.6400,...,0,0,0,0,0,0,0,0,0,1
89737,22,385000,0,0.174,0.1170,0,-18.318,0,0.0401,0.9940,...,0,0,0,0,0,0,0,0,0,1
89738,22,271466,0,0.629,0.3290,0,-10.895,0,0.0420,0.8670,...,0,0,0,0,0,0,0,0,0,1
89739,41,283893,0,0.587,0.5060,7,-10.889,1,0.0297,0.3810,...,0,0,0,0,0,0,0,0,0,1


In [79]:
df.corr()["popularity"].sort_values(ascending=False)

popularity          1.000000
pop-film            0.134407
k-pop               0.122108
chill               0.105387
sad                 0.099098
                      ...   
detroit-techno     -0.113372
latin              -0.127161
instrumentalness   -0.127465
romance            -0.141024
iranian            -0.157932
Name: popularity, Length: 130, dtype: float64